In [7]:
import os
import pandas as pd

"""def segmenta(imagens=1000, dias=5):
    faculdades = ['PUC', 'UFPR04', 'UFPR05']
    tempos = ['Rainy', 'Cloudy', 'Sunny']
    
    num_faculdades = len(faculdades)
    imagens_por_faculdade = [imagens // num_faculdades] * num_faculdades
    for i in range(imagens % num_faculdades):
        imagens_por_faculdade[i] += 1  

    data = []

    for faculdade, num_imagens_faculdade in zip(faculdades, imagens_por_faculdade):
        num_imagens_tempo = [num_imagens_faculdade // len(tempos)] * len(tempos)
        for j in range(num_imagens_faculdade % len(tempos)):
            num_imagens_tempo[j] += 1  

        for tempo, num_imagens_tempo_value in zip(tempos, num_imagens_tempo):
            tempo_path = f"/home/lucas/Downloads/PKLot/PKLotSegmented/{faculdade}/{tempo}"

            if not os.path.exists(tempo_path):
                print(f"O diretório {tempo_path} não existe.")
                continue

            dias_disponiveis = [dia for dia in os.listdir(tempo_path) if os.path.isdir(os.path.join(tempo_path, dia))]

            dias_selecionados = dias_disponiveis[:dias]

            imagens_por_dia = num_imagens_tempo_value // len(dias_selecionados)
            imagens_restantes_dia = num_imagens_tempo_value % len(dias_selecionados)  # Restante para distribuição

            for dia in dias_selecionados:
                dia_path = os.path.join(tempo_path, dia)

                for classe in ['Empty', 'Occupied']:
                    classe_path = os.path.join(dia_path, classe)

                    if not os.path.exists(classe_path):
                        print(f"O diretório {classe_path} não existe.")
                        continue

                    imagens_lista = [os.path.join(classe_path, img) for img in os.listdir(classe_path) if img.endswith('.jpg')]
                    
                    if classe == 'Occupied':
                        num_imagens_classe = imagens_por_dia // 2 + (1 if imagens_restantes_dia > 0 else 0)
                    else:
                        num_imagens_classe = imagens_por_dia // 2

                    print(num_imagens_classe)
                    
                    for img in imagens_lista[:num_imagens_classe]: 
                        data.append({
                            'faculdade': faculdade,
                            'tempo': tempo,
                            'dia': dia,
                            'classe': classe,
                            'imagem': img
                        })

                # Ajustando o restante de imagens
                if imagens_restantes_dia > 0:
                    imagens_restantes_dia -= 1 

    # Criando o DataFrame
    df = pd.DataFrame(data)
    df.to_csv('teste.csv')
    return df

segmenta()"""

'def segmenta(imagens=1000, dias=5):\n    faculdades = [\'PUC\', \'UFPR04\', \'UFPR05\']\n    tempos = [\'Rainy\', \'Cloudy\', \'Sunny\']\n    \n    num_faculdades = len(faculdades)\n    imagens_por_faculdade = [imagens // num_faculdades] * num_faculdades\n    for i in range(imagens % num_faculdades):\n        imagens_por_faculdade[i] += 1  \n\n    data = []\n\n    for faculdade, num_imagens_faculdade in zip(faculdades, imagens_por_faculdade):\n        num_imagens_tempo = [num_imagens_faculdade // len(tempos)] * len(tempos)\n        for j in range(num_imagens_faculdade % len(tempos)):\n            num_imagens_tempo[j] += 1  \n\n        for tempo, num_imagens_tempo_value in zip(tempos, num_imagens_tempo):\n            tempo_path = f"/home/lucas/Downloads/PKLot/PKLotSegmented/{faculdade}/{tempo}"\n\n            if not os.path.exists(tempo_path):\n                print(f"O diretório {tempo_path} não existe.")\n                continue\n\n            dias_disponiveis = [dia for dia in os.l

In [11]:
import os

imagens = 1000
dias = 5

faculdades = ['PUC', 'UFPR04', 'UFPR05']
tempos = ['Rainy', 'Cloudy', 'Sunny']

imagens_por_faculdade = [imagens // len(faculdades)] * len(faculdades)
for i in range(imagens % len(faculdades)):
    imagens_por_faculdade[i] += 1  

data = []

for faculdade, num_imagens_por_faculdade in zip(faculdades, imagens_por_faculdade):
    num_imagens_tempo = [num_imagens_por_faculdade // len(tempos)] * len(tempos)
    for j in range(num_imagens_por_faculdade % len(tempos)):
        num_imagens_tempo[j] += 1

    for tempo, num_imagens_tempo_value in zip(tempos, num_imagens_tempo):
        tempo_path = f"/home/lucas/Downloads/PKLot/PKLotSegmented/{faculdade}/{tempo}"
        if not os.path.exists(tempo_path):
            print(f"O diretório {tempo_path} não existe.")
            continue

        dias_disponiveis = [dia for dia in os.listdir(tempo_path) if os.path.isdir(os.path.join(tempo_path, dia))]
        dias_ordenados = sorted(dias_disponiveis)
        dias_selecionados = dias_ordenados[:dias]

        print(dias_selecionados)

        for dia in dias_selecionados:
            empty_path = os.path.join(tempo_path, dia, 'Empty')
            occupied_path = os.path.join(tempo_path, dia, 'Occupied')

            if os.path.exists(empty_path):
                empty_images = os.listdir(empty_path)
                print(f'Dia {dia}, com o tempo {tempo}, na faculdade {faculdade}, possui : {len(empty_images)} imagens na classe Empty')

            if os.path.exists(occupied_path):
                occupied_images = os.listdir(occupied_path)
                print(f'Dia {dia}, com o tempo {tempo}, na faculdade {faculdade}, possui : {len(occupied_images)} imagens na classe Occupied')

        

#1000 imagens -> PUC:334, UFPR04:333, UFPR05:333 OK
#PUC:334 -> Cloudy:111, Rainy:111, Sunny:112  OK
#Cloudy:111 -> Preencher com 5 dias diferentes OK
#Cloudy:111 -> Pegar os 5 dias de forma ordenada OK
#2012-09-12_06_05_16:22
#2012-09-12_06_05_16:22
#2012-09-12_06_05_16:22
#2012-09-12_06_05_16:22
#2012-09-12_06_05_16:23
#Pegar um dos dias e pegar metade de cada classe -> 2012-09-12_06_05_16:22 / 2 -> Occupied:11, Empty:11
# Caso não tenha 22 imagens em .../PKLotSegmented/{Faculdade}}/{Tempo}/{Dia}/{Classe}/file.jpg -> passar para o proximo dia e pegar até totalizar as imagens balanceadas
#  
#/home/lucas/Downloads/PKLot/PKLotSegmented/PUC/Cloudy/2012-09-12/Empty/2012-09-12_06_05_16#001.jpg

['2012-09-16', '2012-09-21', '2012-10-11', '2012-10-23', '2012-10-25']
Dia 2012-09-16, com o tempo Rainy, na faculdade PUC, possui : 2349 imagens na classe Empty
Dia 2012-09-16, com o tempo Rainy, na faculdade PUC, possui : 51 imagens na classe Occupied
Dia 2012-09-21, com o tempo Rainy, na faculdade PUC, possui : 3685 imagens na classe Empty
Dia 2012-09-21, com o tempo Rainy, na faculdade PUC, possui : 5811 imagens na classe Occupied
Dia 2012-10-11, com o tempo Rainy, na faculdade PUC, possui : 6056 imagens na classe Empty
Dia 2012-10-11, com o tempo Rainy, na faculdade PUC, possui : 12432 imagens na classe Occupied
Dia 2012-10-23, com o tempo Rainy, na faculdade PUC, possui : 76 imagens na classe Empty
Dia 2012-10-23, com o tempo Rainy, na faculdade PUC, possui : 1024 imagens na classe Occupied
Dia 2012-10-25, com o tempo Rainy, na faculdade PUC, possui : 2893 imagens na classe Empty
Dia 2012-10-25, com o tempo Rainy, na faculdade PUC, possui : 10704 imagens na classe Occupied
['2012